## Jack's Car Rental

In [1]:
import numpy as np
import math
import random

In [2]:
max_cars = 20
max_moves = 3
state_ceiling = np.array([max_cars, max_cars])
states = [np.array([i,j]) for i in range(max_cars+1) for j in range(max_cars+1)]

In [3]:
actions = [np.array([-i,i]) for i in range(-max_moves,max_moves+1)]

In [4]:
def poisson(l, k):
    return l**k/math.factorial(k)*math.exp(-l)

In [5]:
p = dict()
p_rentals = [0]*max_cars
p_returns = [0]*max_cars
p_in = [[poisson(3,a)*poisson(2,b) for b in range(max_cars+1)] for a in range(max_cars+1)]
p_out = [[poisson(3,a)*poisson(4,b) for b in range(max_cars+1)] for a in range(max_cars+1)]

In [6]:
m = max_cars+1
p = dict()
for s in states:
    for a in actions:
        p[(s[0],s[1],a[0],a[1])] = dict()

In [7]:
def truncate(s):
    s = np.min([s, state_ceiling], axis=0)
    s = np.max([s, [0,0]], axis = 0)
    return s

In [8]:
import time

In [9]:
IN = 100
OUT = 101
def prob(i, x):
    if (i == IN):
        return p_in[x[0]][x[1]]
    elif (i == OUT):
        return p_out[x[0]][x[1]]
    else:
        return None

In [10]:
lookup_p = dict()
for s_i in states:
    for action in actions:
        s_temp = s_i + action
        if np.any(s_temp < 0) or np.any(s_temp > max_cars):
            continue
        this_p = dict()
        total_p = 0
        for s_f in states:
            w = s_temp - s_f
            for x in range(max(0, w[0]), s_temp[0]+1):
                for y in range(max(0, w[1]), s_temp[1]+1):
                    r = x+y
                    cars_out = np.array([x,y]) # out will always be <= s_temp
                    cars_in = s_f - s_temp + cars_out
                    # print(s_temp,cars_out, cars_in, s_f)
                    # if np.any(cars_in < 0):
                    #    print("Error in cars_in")
                    k = (s_f[0], s_f[1], 10*r)
                    pr = prob(IN, cars_in)*prob(OUT,cars_out)
                    if k in this_p.keys():
                        this_p[k] += pr
                    else:
                        this_p.update( { k : pr } )
                    total_p += pr
        if (total_p > 0):
            for k in this_p:
                this_p[k] /= total_p
        k2 = (s_i[0], s_i[1], action[0], action[1])
        lookup_p.update( { k2: this_p } )

In [ ]:
import pickle
with open("prob-3.pk","wb") as f:
    pickle.dump(lookup_p, f)

# Import data

In [1]:
import pickle
with open("prob-3.pk","rb") as f:
    p = pickle.load(f)

EOFError: Ran out of input